In [ ]:
import scanpy as sc
import os
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import squidpy as sq
import pickle
sc.settings.verbosity=0 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.n_jobs=4
sc.settings.set_figure_params(dpi=900, facecolor="white", frameon=False, figsize=(4,4))
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['pdf.fonttype'] = 42

%matplotlib inline




 

# Name to use

In [ ]:
F1_NAME = 'F1: Superficial'
F23_NAME = 'F2/3: Stroma_PPARG+'
F1_NAME_REGEN= 'F1: Superficial'
F1_REGEN_NAME= 'F1: Superficial'
F6I = "F6: Inflammatory myofibroblast"



In [ ]:
# os.listdir('/nfs/team298/ls34/')

# Load

In [ ]:
adata_5k=sc.read_h5ad('/nfs/team298/ls34/fibroblast_atlas/fig1/adata_xenium_webatlas2.h5ad')
adata_5k

In [ ]:
adata_5k.obs.sample_id.value_counts()

# Map new f5 annotations

In [ ]:
# JUST 2 SAMPLES
#adata_5k=sc.read_h5ad('/nfs/team361/ls34/data/adata_xenium5k_akbar_24dec.twotimepoints5.h5ad.fixed')
#adata_5k.obs.Timepoint.value_counts()

"""A
ORIGINAL DATA
adata_path = '/lustre/scratch126/cellgen/team298/ls34/NicheCompass/ref_only/artifacts/spatial_reference_mapping/20241214_101736BEACON_XENIUM_svg2048_neighbors8_OCT/model/reference_query/adata_final.h5ad'
adata_5k = sc.read_h5ad(adata_path) 
"""

PATH='/nfs/team361/ls34/data/adata_xenium5k_akbar_24dec_allsamples5.h5ad.fixed'
adata_f5=sc.read_h5ad(PATH)
adata_f5.obs.lvl5_annotation.value_counts()


In [ ]:
adata_f5 = adata_f5[adata_f5.obs["lvl5_annotation"].str.startswith("F5")]
adata_f5.obs.lvl5_annotation.value_counts()


In [ ]:
adata_f5.obs.index = [x.split('_')[0] for x in adata_f5.obs.index]
adata_f5.obs.head(5)

In [ ]:
adata_5k.obs.index = [x.split('_')[0] for x in adata_5k.obs.index]
adata_5k.obs.tail(5)


In [ ]:
mapping_dict = adata_f5.obs["lvl5_annotation"].to_dict()
#adata_5k.obs["celltypes_Fig2"] = adata_5k.obs.index.map(mapping_dict).fillna(adata_5k.obs["celltypes_Fig2"] )
##adata_5k.obs["celltypes_Fig2"] = adata_5k.obs.index.to_series().apply(
##    lambda x: mapping_dict.get(x, adata_5k.obs.at[x, "celltypes_Fig2"])
#)
adata_5k.obs["celltypes_Fig2"] = adata_5k.obs.index.to_series().map(mapping_dict).fillna(adata_5k.obs["celltypes_Fig2"])
adata_5k.obs["celltypes_Fig2"].value_counts()




In [ ]:
adata_5k.obs["celltypes_Fig2"].value_counts()


# Figure 2

In [ ]:
sc.settings.figdir = "fig2"


In [ ]:
adata_5k_fig2=adata_5k[adata_5k.obs["sample_id"].isin(["Nonlesional_atopicdermatitis1" , "Nonlesional_atopicdermatitis2"])]
adata_5k_fig2.obs["sample_id"].value_counts()

In [ ]:
desired_order = [
     F1_NAME, 'F2: Universal', F23_NAME, 'F3: FRC-like', 'F4: DS_DPEP1+', 'F4: TNN+COCH+',
    'F5: Schwann-like', 'F5: NGFR+', 'F5: RAMP1+',
    'Schwann','VE/Pericyte',  'Immune cell', 'Other'
]

adata_5k.obs['celltypes_Fig2'] = pd.Categorical(adata_5k.obs['celltypes_Fig2'], categories=desired_order, ordered=True)

adata_5k.obs["celltypes_Fig2"] =  adata_5k.obs["celltypes_Fig2"].astype('category')

# color_map_5k = {
#     'F3: FRC-like': '#FF0000', 
#     F23_NAME: '#0000FF', 

#     'F5: Schwann-like': "#7850b2", 
#     'VE/Pericyte': '#696969', 
#     'F2: Universal': '#ADD8E6',
    
#     F1_NAME: '#FFFF00',    'F1: Regenerative': '#FFFF00',

#     'Schwann': '#000000',
#         'F4: DS_DPEP1+': '#008000',          
#     'F4: TNN+COCH+': '#80ff80',     
#     'Other': '#DCDCDC', 
    
#     'Immune cell': "#D2B48C"
# }
# adata_5k.uns['celltypes_Fig2_colors'] = [color_map_5k[x] for x in adata_5k.obs["celltypes_Fig2"].cat.categories]


In [ ]:
sc.settings.set_figure_params(dpi=100, facecolor="white", frameon=False, figsize=(18,18))
"""
all samples - broad immune cells (more specific immune cells below)
"""
# CATEGORY = "broad_annotation"
# KEEP_5k =[x for x in adata_5k.obs[CATEGORY].unique() if x.startswith("F") or x.startswith("Schw") or x.startswith("VE")
#           or x.startswith("Immun")
#          ]
# KEEP_5k
# adata_5k.obs["fig2"] = [x if x in KEEP_5k else "Other" for x in adata_5k.obs[CATEGORY]]

# Define the desired order for the legend
# desired_order = [
#      F1_NAME, 'F2: Universal', F23_NAME, 'F3: FRC-like', 'F4: DS_DPEP1+', 'F4: TNN+COCH+',
#     'F5: Schwann-like', 'Schwann', 'Immune cell', 'VE/Pericyte', 'Other'
# ]

# Set the categorical order for `fig2`
#adata_5k.obs['celltypes_Fig2'] = pd.Categorical(adata_5k.obs['fig2'], categories=desired_order, ordered=True)

adata_5k.obs["celltypes_Fig2"] =  adata_5k.obs["celltypes_Fig2"].astype('category')


color_map_5k = {
    'F3: FRC-like': "#FF1493",#  '#FF0000', 
    F23_NAME: '#0000FF',
    #'F2 Universal/F1': '#4d4d4d',

    'F5: Schwann-like': "#7850b2", #'#800080',
    'VE/Pericyte':"#FFC0CB",# "#8B0000", "#FFC1C1", #'#696969', #'#A9A9A9',  "#FF0000", 
    'F2: Universal': '#ADD8E6',
    #'F2 Universal/F1': '#0000FF',
    
    "F1: Superficial": '#FFFF00',    'F1: Regenerative': '#FFFF00',

    'Schwann': '#000000',
    
    # Updated F4 colors
    'F4: DS_DPEP1+': '#008000',         # Dark green
    #'F4: Mixed': '#',      # Mid green (standard green)
    'F4: TNN+COCH+': '#80ff80',     # Very light green
    
    'Other': '#DCDCDC', #'lightgrey'
    
    'Immune cell': "#D2B48C"
}
adata_5k.uns['celltypes_Fig2_colors'] = [color_map_5k[x] for x in adata_5k.obs["celltypes_Fig2"].cat.categories]


#adata_5k2 = adata_5k[adata_5k.obs['run'] =="oct"]
import squidpy as sq
CATEGORY = "celltypes_Fig2"
adata_5k=adata_5k[adata_5k.obs[CATEGORY]!="EXCLUDE"]

sc.settings.set_figure_params(dpi=300, facecolor="white", frameon=False, figsize=(13,13))

# for SAMPLE_INDEX, sample in enumerate( adata_5k.obs["sample_id"] .unique()):
#     print(sample)
# # if "Week 12" in sample:
#    # adata_i_f_5k=adata_5k[adata_5k.obs["info_id"]=="BK18_Non-lesional Baseline"]

#     adata_i_f_5k = adata_5k[adata_5k.obs["sample_id"] == sample]
#     print(adata_i_f_5k.shape)
# #     try:
# #         adata_i_f_5k.uns.pop('celltypes_Fig2_')
# #     except:
# #         1  


#    # adata_i_f_5k.obs["fig2"] =  adata_i_f_5k.obs["fig2"].astype('category')

#     #print(adata_i_f_5k.obs["specific_celltype4"].unique())
#     # Generate the plots
#     sq.pl.spatial_scatter(
#         adata_i_f_5k,
#         library_id="spatial",
#         shape=None,
#         color="celltypes_Fig2",
#         title=sample  ,
#         size=30,
#         linewidth=0.1,
#         edgecolor='black'
#         #save="_fig2umap1a.png"
#     )
#     sq.pl.spatial_scatter(
#         adata_i_f_5k,
#         library_id="spatial",
#         shape=None,
#         color="celltypes_Fig2",
#         title=sample  ,
#         size=50,
#                 linewidth=0.01,
#         edgecolor='black'
#         #save="_fig2umap1b.png"
#     )


# for SAMPLE_INDEX, sample in enumerate( [  ]):
#     print(sample)
# # if "Week 12" in sample:
#    # adata_i_f_5k=adata_5k[adata_5k.obs["info_id"]=="BK18_Non-lesional Baseline"]

#     adata_i_f_5k = adata_5k[adata_5k.obs["info_id"] == LIST_OF_SAMPLES[SAMPLE_INDEX]]
#     print(adata_i_f_5k.shape)
#     try:
#         adata_i_f_5k.uns.pop('specific_celltype4_colors')
#     except:
#         1  


#    # adata_i_f_5k.obs["fig2"] =  adata_i_f_5k.obs["fig2"].astype('category')

#     #print(adata_i_f_5k.obs["specific_celltype4"].unique())
#     # Generate the plots
#     sq.pl.spatial_scatter(
#         adata_i_f_5k,
#         library_id="spatial",
#         shape=None,
#         color="fig2",
#         title=sample  ,
#         size=10,
#         save="_fig2umap2.png"
#     )
sc.settings.set_figure_params(dpi=50, facecolor="white", frameon=False, figsize=(5,5))



In [ ]:
sc.settings.set_figure_params(dpi=100, facecolor="white", frameon=False, figsize=(18,18))
"""
all samples - broad immune cells (more specific immune cells below)
"""


CATEGORY = "celltypes_Fig2"
#adata_5k=adata_5k[adata_5k.obs[CATEGORY]!="EXCLUDE"]

sc.settings.set_figure_params(dpi=300, facecolor="white", frameon=False, figsize=(13,13))

for SAMPLE_INDEX, sample in enumerate( [ "Nonlesional_atopicdermatitis1" ]):
    print(sample)

    adata_i_f_5k = adata_5k[adata_5k.obs["sample_id"] == sample]
    print(adata_i_f_5k.shape)
    try:
        adata_i_f_5k.uns.pop('specific_celltype4_colors')
    except:
        1  

    sq.pl.spatial_scatter(
        adata_i_f_5k,
        library_id="spatial",
        shape=None,
        color="celltypes_Fig2",
        title=sample  ,
        size=25,
         linewidth=0.05,
         edgecolor='black'
       # save="_fig2umap1b.png"
    )
    sq.pl.spatial_scatter(
        adata_i_f_5k,
        library_id="spatial",
        shape=None,
        color="celltypes_Fig2",
        title=sample  ,
        size=40,
         linewidth=0.1,
         edgecolor='grey'
       # save="_fig2umap1b.png"
    )
    
for SAMPLE_INDEX, sample in enumerate( [ "Nonlesional_atopicdermatitis2" ]):
    print(sample)
    adata_i_f_5k = adata_5k[adata_5k.obs["sample_id"] == sample]
    print(adata_i_f_5k.shape)
    try:
        adata_i_f_5k.uns.pop('specific_celltype4_colors')
    except:
        1  

    sq.pl.spatial_scatter(
        adata_i_f_5k,
        library_id="spatial",
        shape=None,
        color="celltypes_Fig2",
        title=sample  ,
        size=25,
             linewidth=0.05,
         edgecolor='black'
      #  save="_fig2umap1a.png"
    )
    sq.pl.spatial_scatter(
        adata_i_f_5k,
        library_id="spatial",
        shape=None,
        color="celltypes_Fig2",
        title=sample  ,
        size=40,
         linewidth=0.1,
         edgecolor='grey'
       # save="_fig2umap1b.png"
    )
    

 
   
sc.settings.set_figure_params(dpi=50, facecolor="white", frameon=False, figsize=(5,5))



# sfig2

In [ ]:
sc.settings.figdir = "sfig2"


In [ ]:
file_path = '/nfs/team298/ls34/fb_dict.pkl'

with open(file_path, 'rb') as file:
    reynolds_short_final = pickle.load(file)

reynolds_short_final = {key: [gene for gene in genes if gene in adata_5k_fig2.var_names] 
                           for key, genes in reynolds_short_final.items()}

adata_5k_fig2_withfb = adata_5k_fig2[adata_5k_fig2.obs["celltypes_Fig2"].str.startswith("F")]

sc.pl.dotplot(
    adata_5k_fig2_withfb,
    reynolds_short_final,
    groupby="celltypes_Fig2",
    standard_scale="var",
    dendrogram=False,
    colorbar_title="Mean expression\n(variance scaled)",
    categories_order=  ['F1: Superficial', 'F2: Universal', 'F2/3: Stroma_PPARG+', 'F3: FRC-like', 'F4: DS_DPEP1+', 'F4: TNN+COCH+',
                        'F5: Schwann-like',]

) 
sc.pl.matrixplot(
    adata_5k_fig2_withfb,
    reynolds_short_final,
    groupby="celltypes_Fig2",
   standard_scale="var",
    vmax=1,cmap="Reds",
    dendrogram=False,
    
    colorbar_title="Mean expression\n(variance scaled)",
    categories_order=  ['F1: Superficial', 'F2: Universal', 'F2/3: Stroma_PPARG+', 'F3: FRC-like', 'F4: DS_DPEP1+', 'F4: TNN+COCH+',
                        'F5: Schwann-like',]

) 


In [ ]:
FIG6_GENES= {   "Schwann": ["ITGA6", "SCN7A"]   ,       
                "Immune cell": ["CCR7", "LAMP3", "CD1B", "CD163", "CD3E", "GATA3", "CXCR4", 'CD96'],
                    "VE/Peri": ["CLDN5", "PECAM1", "SOX18", "RGS5"],

              }
adata_5k_fig2_withoutfb = adata_5k_fig2[~adata_5k_fig2.obs["celltypes_Fig2"].str.startswith("F")]
 
sc.pl.dotplot(
    adata_5k_fig2_withoutfb,
    FIG6_GENES,
    groupby="celltypes_Fig2",
    standard_scale="var",
    dendrogram=False,
   #swap_axes=True,
    colorbar_title="Mean expression\n(variance scaled)"
) 
sc.pl.matrixplot(
    adata_5k_fig2_withoutfb,
    FIG6_GENES,
    groupby="celltypes_Fig2",
    standard_scale="var",cmap="Reds",
    dendrogram=False,
   #swap_axes=True,
    colorbar_title="Mean expression\n(variance scaled)"
) 

In [ ]:
adata_5k_fig2.obs["sample_id"].value_counts()
adata_5k_fig2_i=adata_5k_fig2[adata_5k_fig2.obs["sample_id"]=="Nonlesional_atopicdermatitis2"]

sq.gr.spatial_neighbors(adata_5k_fig2_i, 
                             spatial_key='spatial', #elements_to_coordinate_systems=None, table_key=None, library_key=None, coord_type=None, 
                             #n_neighs=4,
                            radius=20)# radius=None, 
#adata_5k_perivasc = adata_5k[adata_5k.obs["niche_name"]=="Perivascular_T-rich"]
#adata_5k_perivasc = adata_5k_perivasc[adata_5k_perivasc.obs["Timepoint"]=="Lesional Baseline"]



sq.pl.spatial_scatter(
    adata_5k_fig2_i,
    shape=None,
    color="celltypes_Fig2",
    connectivity_key="spatial_connectivities",
    size=1,
)
sq.gr.nhood_enrichment(adata_5k_fig2_i, cluster_key="celltypes_Fig2")
sq.pl.nhood_enrichment(
    adata_5k_fig2_i,
    cluster_key="celltypes_Fig2",
    #figsize=(8, 8),
    title="Neighborhood enrichment adata",
    vmax=10,
    vmin=-10,
    cmap="Reds", #"bwr",
    #font_size=4,
    figsize=(20,20),
    #ax=ax[0],
)


In [ ]:
import scanpy as sc
import squidpy as sq
import matplotlib.pyplot as plt

# # Your custom color map
# color_map_5k = {
#     'F3: FRC-like': "#FF1493",  # Neon pink
#     F23_NAME: '#0000FF',        # Blue
#     'F5: Schwann-like': "#7850b2",  # Purple
#     'VE/Pericyte': "#FFC1C1",   # Light pink
#     'F2: Universal': '#ADD8E6', # Light blue
#     'F1: Superficial': '#FFFF00', 
#     'F1: Regenerative': '#FFFF00', # Yellow
#     'Schwann': '#000000',       # Black
#     'F4: DS_DPEP1+': '#008000', # Dark green
#     'F4: TNN+COCH+': '#80ff80', # Light green
#     'Other': '#DCDCDC',         # Light grey
#     'Immune cell': "#D2B48C"    # Tan
# }

# # Ensure the order of colors matches the cluster categories
# cluster_categories = adata_5k_fig2_i.obs["celltypes_Fig2"].cat.categories
# colors_in_order = [color_map_5k.get(cat, '#808080') for cat in cluster_categories]  # Default to grey if missing

# # Assign colors to the adata object
# adata_5k_fig2_i.uns["celltypes_Fig2_colors"] = colors_in_order

# Plot neighborhood enrichment with custom colors
sq.pl.nhood_enrichment(
    adata_5k_fig2_i,
    cluster_key="celltypes_Fig2",
    title="Neighborhood enrichment adata",
    vmax=10,
    vmin=-10,
    cmap="Reds",  # This still controls the heatmap values
    figsize=(20, 20),
)

In [ ]:
adata_5k_fig2.obs["sample_id"].value_counts()
adata_5k_fig2_i=adata_5k_fig2[adata_5k_fig2.obs["sample_id"]!="Nonlesional_atopicdermatitis2"]

In [ ]:
sq.gr.spatial_neighbors(adata_5k_fig2_i, 
                             spatial_key='spatial', #elements_to_coordinate_systems=None, table_key=None, library_key=None, coord_type=None, 
                             n_neighs=4,
                           # radius=20
                       )# radius=None, 
#adata_5k_perivasc = adata_5k[adata_5k.obs["niche_name"]=="Perivascular_T-rich"]
#adata_5k_perivasc = adata_5k_perivasc[adata_5k_perivasc.obs["Timepoint"]=="Lesional Baseline"]



sq.pl.spatial_scatter(
    adata_5k_fig2_i,
    shape=None,
    color="celltypes_Fig2",
    connectivity_key="spatial_connectivities",
    size=1,
)
sq.gr.nhood_enrichment(adata_5k_fig2_i, cluster_key="celltypes_Fig2")
sq.pl.nhood_enrichment(
    adata_5k_fig2_i,
    cluster_key="celltypes_Fig2",
    #figsize=(8, 8),
    title="Neighborhood enrichment adata",
    vmax=10,
    vmin=-10,
    cmap="Reds", #"bwr",
    #font_size=4,
    figsize=(20,20),
    #ax=ax[0],
)


# supp figure (marker gene specificity)

In [ ]:
GENES = ['COL23A1', 'CD70', 'PPARG', 'CCL19', 'COCH', 'IGFBP2', 'SFRP4']


adata_5k_fig2_withfb = adata_5k_fig2[adata_5k_fig2.obs["celltypes_Fig2"].str.startswith("F")]

sc.pl.matrixplot(
    adata_5k_fig2_withfb,
    GENES,
    groupby="celltypes_Fig2",
    standard_scale="var",
    dendrogram=False,
    cmap="Reds",
    #dot_max=0.5,
    colorbar_title="Mean expression\n(variance scaled)",
    categories_order=  ['F1: Superficial', 'F2: Universal', 'F2/3: Stroma_PPARG+', 'F3: FRC-like', 'F4: DS_DPEP1+', 'F4: TNN+COCH+',
                        'F5: Schwann-like',]
  
) 
sc.pl.matrixplot(
    adata_5k_fig2,
    GENES,
    groupby="celltypes_Fig2",
   standard_scale="var",
    vmax=1,#cmap="Reds",
    dendrogram=False, 
    cmap="Reds",
    
    colorbar_title="Mean expression\n(variance scaled)",
   categories_order=  ['F1: Superficial', 'F2: Universal', 'F2/3: Stroma_PPARG+', 'F3: FRC-like', 'F4: DS_DPEP1+', 'F4: TNN+COCH+',
                    'F5: Schwann-like', "Schwann", "Immune cell", "VE/Pericyte", "Other"] 

) 


# fig 4

In [ ]:
sc.settings.figdir = "fig4"


In [ ]:
adata_eczema = adata_5k[adata_5k.obs["sample_id"] == "Lesional_atopicdermatitis3"]

category_counts = adata_eczema.obs["celltypes_Fig4"].value_counts()

CATEGORY = "celltypes_Fig4"
KEEP_5k =[x for x in adata_5k.obs[CATEGORY].unique() if x.startswith("F") or x.startswith("Schw") or x.startswith("VE")
          or x.startswith("Immun")
         ]
KEEP_5k
adata_5k.obs["celltypes_Fig4"] = [x if x in KEEP_5k else "Other" for x in adata_5k.obs[CATEGORY]]

adata_5k.obs["celltypes_Fig4"].value_counts()
 

In [ ]:
sc.settings.set_figure_params(dpi=300, facecolor="white", frameon=False, figsize=(13,13))

for SAMPLE_INDEX, sample in enumerate(["Lesional_atopicdermatitis3"]):
    print(sample)
    if "Lesional" in sample:
        try:
            adata_eczema.uns.pop('celltypes_Fig4_colors')
        except:
            1 
        sq.pl.spatial_scatter(
            adata_eczema,
            library_id="spatial",
            shape=None,
            color="celltypes_Fig2",
            title=sample,
            size=5,
            #save="_fig6umap_xenium.png"
        )


    if "Lesional" in sample:
        try:
            adata_eczema.uns.pop('celltypes_Fig4_colors')
        except:
            1 
        sq.pl.spatial_scatter(
            adata_eczema,
            library_id="spatial",
            shape=None,
            color="celltypes_Fig2",
            title=sample,
            size=25,
            legend_loc=None,
                     linewidth=0.1,
         edgecolor='grey'
            #save="_fig6umap_xenium.png"
        )

    if "Lesional" in sample:
        try:
            adata_eczema.uns.pop('celltypes_Fig4_colors')
        except:
            1 
        sq.pl.spatial_scatter(
            adata_eczema,
            library_id="spatial",
            shape=None,
            color="celltypes_Fig2",
            title=sample,
            size=25,
            legend_loc=None,
                     linewidth=0.05,
         edgecolor='black'
            #save="_fig6umap_xenium.png"
        )

     

In [ ]:
sc.settings.set_figure_params(dpi=300, facecolor="white", frameon=False, figsize=(13,13))

for SAMPLE_INDEX, sample in enumerate(["Lesional_atopicdermatitis3"]):
    print(sample)


    if "Lesional" in sample:
        try:
            adata_eczema.uns.pop('celltypes_Fig4_colors')
        except:
            1 
        sq.pl.spatial_scatter(
            adata_eczema,
            library_id="spatial",
            shape=None,
            color="celltypes_Fig2",
            title=sample,
            size=25,
            legend_loc=None,
                     linewidth=0.1,
         edgecolor='black'
            #save="_fig6umap_xenium.png"
        )

     
    if "Lesional" in sample:
        try:
            adata_eczema.uns.pop('celltypes_Fig4_colors')
        except:
            1 
        sq.pl.spatial_scatter(
            adata_eczema,
            library_id="spatial",
            shape=None,
            color="celltypes_Fig2",
            title=sample,
            size=25,
            legend_loc=None,
                     linewidth=0.05,
         edgecolor='black'
            #save="_fig6umap_xenium.png"
        )

     

In [ ]:
sc.settings.figdir = "sfig4"


In [ ]:
reynolds_short_final={'F1': ['APCDD1', 'COL18A1', 'COL23A1', 'COMP', 'AXIN2'],
 "F1_activated": [ 'TNFRSF21', 'WNT5A', 'COL7A1'],
 'F2: Universal': ['CD34', 'PI16', 'DPP4', 'MFAP5', 'CTHRC1', 'CD70', 'LGR5'],
 'F2/3: Bridge': ['CXCL12', 'PPARG'],
 'F3': ['CCL19', 'CH25H', 'TNFSF13B', 'IL33', 'IRF8', 'IL15', 'VCAM1'],
 "F3: Activated": ["ADAMDEC1", "CXCL9"],
 'F4': ['ASPN'],
 'F4: DS_DPEP1+': ['COL11A1', 'MEF2C'],
 'F4: TNN+COCH+': ['COCH', 'MKX', 'TNMD'],
 'F4: DP_HHIP+': ['CORIN', 'BMP7', 'WNT5A', 'LEF1', 'RSPO3', 'INHBA', 'PTCH1'],
 'F5': ['SCN7A', 'FGFBP2'],
 'F5+: RAMP1': ['RELN', 'IGFBP2', 'SFRP1'],
 'F5: NGFR+': ['EBF2', 'NGFR', 'SFRP4', 'ITGA6', 'CLDN1']}

adata_eczema2 = adata_eczema[adata_eczema.obs["celltypes_Fig2"].str.startswith("F")]

sc.pl.dotplot(
    adata_eczema2,
    reynolds_short_final,
    groupby="celltypes_Fig2",
    standard_scale="var",
    dendrogram=False,
    colorbar_title="Mean expression\n(variance scaled)",
    categories_order=  ['F1: Superficial', 'F2: Universal', 'F2/3: Stroma_PPARG+', 'F3: FRC-like', #'F4: DS_DPEP1+', 'F4: TNN+COCH+',
                        'F5: Schwann-like',]

) 
sc.pl.matrixplot(
    adata_eczema2,
    reynolds_short_final,
    groupby="celltypes_Fig2",
   standard_scale="var",
    vmax=1,cmap="Reds",
    dendrogram=False,
    
    colorbar_title="Mean expression\n(variance scaled)",
    categories_order=  ['F1: Superficial', 'F2: Universal', 'F2/3: Stroma_PPARG+', 'F3: FRC-like', #'F4: DS_DPEP1+', 'F4: TNN+COCH+',
                        'F5: Schwann-like',]

) 


kirstin= {   "Schwann": ["ITGA6", "SCN7A"]   ,       
                "Immune cell": ["CCR7", "LAMP3", "CD1B", "CD163", "CD3E", "GATA3", "CXCR4", 'CD96'],
                    "VE/Peri": ["CLDN5", "PECAM1", "SOX18", "RGS5"],

              }
kirstin = {key: [gene for gene in genes if gene in adata_i_f_5k.var_names] 
                           for key, genes in kirstin.items()}


adata_eczema2 = adata_eczema[~adata_eczema.obs["celltypes_Fig2"].str.startswith("F")]
sc.pl.matrixplot(
    adata_eczema2,
    kirstin,
    groupby="celltypes_Fig2",
   standard_scale="var",
    vmax=1,cmap="Reds",
    dendrogram=False,
    
    colorbar_title="Mean expression\n(variance scaled)",
 

) 
 

In [ ]:
sc.settings.figdir = "fig4"


In [ ]:
custom_colors ={'F2: Universal': [0.81411765, 0.88392157, 0.94980392, 1.        ],
                F1_NAME: [1.        , 0.97244137, 0.75680123, 1.        ],
                  'F1: Regenerative': [1.        , 0.97244137, 0.75680123, 1.        ],
                'F3: FRC-like': [0.99692426, 0.89619377, 0.84890427, 1.        ],
                'F4': [0.50196078, 0.79058824, 0.50143791, 1.        ],
                F23_NAME: [0.47294118, 0.71163399, 0.85071895, 1.        ],
                'F5: Schwann-like': [0.56608997, 0.55218762, 0.75933872, 1.        ]

               }


adata_eczema_fbs = adata_eczema[adata_eczema.obs["celltypes_Fig2"].str.startswith("F")]
proportions = adata_eczema_fbs.obs['celltypes_Fig2'].value_counts(normalize=True)

proportions_filtered = proportions[proportions.index != 'Nonspecific']


fig, ax = plt.subplots(figsize=(6, 0.5))



previous_proportion = 0
for proportion, label in zip(proportions_filtered, proportions_filtered.index):
    color = custom_colors.get(label, '#D3D3D3')  # Default color for 'Other'
    ax.barh([1], [proportion], left=previous_proportion, color=color, label=label,
            linewidth=0.5,   
         edgecolor='black',  )
    previous_proportion += proportion

handles, labels = ax.get_legend_handles_labels()
unique_labels = dict(zip(labels, handles))  # Remove duplicates
ax.legend(unique_labels.values(), unique_labels.keys(), loc='lower center', bbox_to_anchor=(0.5, 1.2),
          ncol=len(unique_labels), frameon=False)

ax.set_yticks([])
ax.set_xticks([0, 0.25, 0.5, 0.75, 1.0])
ax.set_xticklabels(['0', '0.25', '0.5', '0.75', '1'])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax.set_ylabel('')
ax.set_xlabel('Proportion')
ax.grid(False)
plt.tight_layout()
plt.show()
 

In [ ]:
desired_order = [F1_NAME, 'F2: Universal', F23_NAME, 'F3: FRC-like', 'F5: Schwann-like']
desired_order = desired_order[::-1]

proportions_filtered = proportions_filtered.reindex(desired_order)

fig, ax = plt.subplots(figsize=(1, 6))  

previous_proportion = 0
for proportion, label in zip(proportions_filtered, proportions_filtered.index):
    color = custom_colors.get(label, '#D3D3D3')   
    ax.bar([1], [proportion], bottom=previous_proportion, color=color, label=label,
                   linewidth=0.5,   
        width=0.5,        edgecolor='black',  

           
          )
    previous_proportion += proportion

handles, labels = ax.get_legend_handles_labels()
unique_labels = dict(zip(labels, handles))  
ax.legend(unique_labels.values(), unique_labels.keys(), loc='upper left', bbox_to_anchor=(1, 1),
          frameon=False)

ax.set_xticks([])
ax.set_yticks([0, 0.25, 0.5, 0.75, 1.0])
ax.set_yticklabels(['0', '0.25', '0.5', '0.75', '1'])
ax.set_ylabel('Proportion')
ax.set_xlabel('')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax.grid(False)
plt.tight_layout()
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), bbox_to_anchor=(1.05, 1), loc='upper left', frameon=False, fontsize=12)
plt.show()

 

# MELANOMA

In [ ]:
adata=adata_5k[adata_5k.obs["sample_id"]=="Public_cutaneousmelanoma"].copy()


In [ ]:

mm_colors =  {
        'Melanoma': '#5C4033',   
     F6I: '#01ffff',  
      'F6: Myofibroblast': '#55aaff',   
    "Bcell/plasma cell": "#FF69B4"

}
color_map_5k = color_map_5k | mm_colors




In [ ]:
CATEGORY="celltypes_Fig4_melanoma"
try:
    adata.uns.pop(CATEGORY + "_colors")
except:
    1 
adata.obs[CATEGORY] =  adata.obs[CATEGORY].astype('category')
adata.uns[CATEGORY+"_colors"] = [color_map_5k[x] for x in adata.obs[CATEGORY].cat.categories]

sq.pl.spatial_scatter(
    adata,
    library_id="spatial",
    shape=None,
    color=CATEGORY,
    title='',
    size=5,
    edgecolor="black",
    linewidth=0.1
    #save="fig4_melanoma.png"
)

In [ ]:
custom_colors ={'F2: Universal': [0.81411765, 0.88392157, 0.94980392, 1.        ],
                F1_NAME: [1.        , 0.97244137, 0.75680123, 1.        ],
                  'F1: Regenerative': [1.        , 0.97244137, 0.75680123, 1.        ],
                'F3: FRC-like': [0.99692426, 0.89619377, 0.84890427, 1.        ],
                'F4': [0.50196078, 0.79058824, 0.50143791, 1.        ],
                F23_NAME: [0.47294118, 0.71163399, 0.85071895, 1.        ],
                'F5: Schwann-like': [0.56608997, 0.55218762, 0.75933872, 1.        ],
                F6I:     '#01ffff',  # cyan
                "F6: Myofibroblast":  '#55aaff',  # light blue

               }


adata_mm_fbs = adata[adata.obs[CATEGORY].str.startswith("F")]
proportions = adata_mm_fbs.obs[CATEGORY].value_counts(normalize=True)

proportions_filtered = proportions[proportions.index != 'Nonspecific']


fig, ax = plt.subplots(figsize=(6, 0.5))



previous_proportion = 0
for proportion, label in zip(proportions_filtered, proportions_filtered.index):
    color = custom_colors.get(label, '#D3D3D3')  
    ax.barh([1], [proportion], left=previous_proportion, color=color, label=label)
    previous_proportion += proportion


handles, labels = ax.get_legend_handles_labels()
unique_labels = dict(zip(labels, handles))   
ax.legend(unique_labels.values(), unique_labels.keys(), loc='lower center', bbox_to_anchor=(0.5, 1.2),
          ncol=len(unique_labels), frameon=False)


ax.set_yticks([])
ax.set_xticks([0, 0.25, 0.5, 0.75, 1.0])
ax.set_xticklabels(['0', '0.25', '0.5', '0.75', '1'])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax.set_ylabel('')
ax.set_xlabel('Proportion')

ax.grid(False)

plt.tight_layout()

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(1, 6))   

previous_proportion = 0
for proportion, label in zip(proportions_filtered, proportions_filtered.index):
    color = custom_colors.get(label, '#D3D3D3')  
    ax.bar([1], [proportion], bottom=previous_proportion, color=color, label=label,
                   linewidth=0.5,   
        width=0.5,        edgecolor='black', 
           

           
          )
    previous_proportion += proportion

    handles, labels = ax.get_legend_handles_labels()
unique_labels = dict(zip(labels, handles))   
ax.legend(unique_labels.values(), unique_labels.keys(), loc='upper left', bbox_to_anchor=(1, 1),
          frameon=False)

ax.set_xticks([])
ax.set_yticks([0, 0.25, 0.5, 0.75, 1.0])
ax.set_yticklabels(['0', '0.25', '0.5', '0.75', '1'])
ax.set_ylabel('Proportion')
ax.set_xlabel('')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax.grid(False)

plt.tight_layout()

handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), bbox_to_anchor=(1.05, 1), loc='upper left', frameon=False, fontsize=12)

plt.show()



In [ ]:
sc.settings.figdir = "sfig4"


In [ ]:
CELL_TYPE="Bcell/plasma cell"



custom_palette = {
    CELL_TYPE: "#00008B",  
    "Other": "#D3D3D3"  
}

adata.obs["fig6_colored"] = [x if x == CELL_TYPE else "Other" for x in adata.obs["celltypes_sFig4_melanoma"]]

adata.obs["fig6_colored"] .value_counts()
adata.obs["fig6_colored"]=adata.obs["fig6_colored"].astype('category')

adata.uns['fig6_colored_colors'] = [custom_palette[cat] for cat in adata.obs["fig6_colored"].cat.categories]


sq.pl.spatial_scatter(
    adata,
    library_id="spatial",
    shape=None,
    color="fig6_colored",
    title=sample,
    size=1
)



In [ ]:
CELL_TYPE="Melanoma"



custom_palette = {
    CELL_TYPE:  "#5C4033",  
    "Other": "#D3D3D3"   
}
adata.obs["fig6_colored"] = [x if x == CELL_TYPE else "Other" for x in adata.obs["celltypes_sFig4_melanoma"]]

adata.obs["fig6_colored"] .value_counts()
adata.obs["fig6_colored"]=adata.obs["fig6_colored"].astype('category')
adata.uns['fig6_colored_colors'] = [custom_palette[cat] for cat in adata.obs["fig6_colored"].cat.categories]

sq.pl.spatial_scatter(
    adata,
    library_id="spatial",
    shape=None,
    color="fig6_colored",
    title=sample,
    size=1,
   # save="umap_melanomaALONE.png"
)



In [ ]:
CELL_TYPE="T"

custom_palette = {
    CELL_TYPE: "#00008B",  
    "Other": "#D3D3D3"   
}

adata.obs["fig6_colored"] = [x if x == CELL_TYPE else "Other" for x in adata.obs["celltypes_sFig4_melanoma"]]

adata.obs["fig6_colored"] .value_counts()
adata.obs["fig6_colored"]=adata.obs["fig6_colored"].astype('category')
adata.uns['fig6_colored_colors'] = [custom_palette[cat] for cat in adata.obs["fig6_colored"].cat.categories]

sq.pl.spatial_scatter(
    adata,
    library_id="spatial",
    shape=None,
    color="fig6_colored",
    title=sample,
    size=5
)



In [ ]:
import pickle

file_path = '/nfs/team298/ls34/fb_dict_lesional.pkl'

with open(file_path, 'rb') as file:
    lesional_genes = pickle.load(file)

print(f"Dictionary successfully LOADED from {file_path}")

lesional_genes = {key: [gene for gene in genes if gene in adata_5k_fig2.var_names] 
                           for key, genes in lesional_genes.items()}
lesional_genes

In [ ]:
adata_mm = adata[adata.obs["celltypes_Fig4_melanoma"].str.startswith("F")]

sc.pl.matrixplot(
    adata_mm,
    lesional_genes,
    groupby="celltypes_sFig4_melanoma",
   #standard_scale="var",
    vmax=0.3,cmap="Reds",
    dendrogram=False,
    
    colorbar_title="Mean expression\n(log1p norm)",
    categories_order=  ['F1: Superficial','F6: Inflammatory myofibroblast',  'F6: Myofibroblast', ])


In [ ]:
lesional_genes = {'F3': ['CCL19', 'CH25H'],
 'F1': ['COL18A1', 'COL23A1', 'COMP', 'TNFRSF21'],
 'F1+F6:My_Inf': ['WNT5A', 'COL7A1'],
 'F6: Myofibroblast inflammatory': ['CXCL5',
  'CXCL6',
  'CXCL13',
  'IL11',
  'IL24',
  'INHBA',
  'CSF3',
  'TDO2',
  'WWC1',
  'STAT4',
  'CCL11'],
 'F6 (both)': ['WNT2', 'COL10A1'],
 'F6(both)+F7': ['ASPN',
  'ZNF469',
  'RUNX2',
  'SULF1',
  'ADAM12',
  'LRRC15',
  'CTHRC1',
  'CCN4',
  'CDH2'],
 'F6: MyoFb + F7': ['PIEZO2', 'SFRP4', 'KCNMA1'],
                 }
adata_mm = adata[adata.obs["celltypes_Fig4_melanoma"].str.startswith("F")]


sc.pl.matrixplot(
    adata_mm,
    lesional_genes,
    groupby="celltypes_sFig4_melanoma",
   #standard_scale="var",
    vmax=0.3,cmap="Reds",
    dendrogram=False,
    
    colorbar_title="Mean expression\n(log1p norm)",
    categories_order=  ['F1: Superficial','F6: Inflammatory myofibroblast',  'F6: Myofibroblast', ])


In [ ]:
adata_mm = adata[adata.obs["celltypes_Fig4_melanoma"].str.startswith("F")]


sc.pl.matrixplot(
    adata_mm,
    lesional_genes,
    groupby="celltypes_sFig4_melanoma",
   #standard_scale="var",
    vmax=0.3,cmap="Reds",
    dendrogram=False,
    
    colorbar_title="Mean expression\n(log1p norm)",
    categories_order=  ['F1: Superficial', 'F6: Inflammatory myofibroblast',  'F6: Myofibroblast', ])

sc.pl.matrixplot(
    adata_mm,
    lesional_genes,
    groupby="celltypes_sFig4_melanoma",
   standard_scale="var",
    vmax=1,cmap="Reds",
    dendrogram=False,
    
    colorbar_title="Mean expression\n(variance scaled)",
    categories_order=  ['F1: Superficial', 'F6: Inflammatory myofibroblast',  'F6: Myofibroblast', ]

) 



In [ ]:
adata_mm = adata[~adata.obs["celltypes_sFig4_melanoma"].str.startswith("F")]
adata_mm = adata_mm[(adata_mm.obs["celltypes_sFig4_melanoma"].str.startswith("B"))|
        (adata_mm.obs["celltypes_sFig4_melanoma"].str.startswith("M"))]


 


BCELL_MM_GENES = {"Bcell/plasma cell": ["MZB1", "CD79A", "POU2AF1", "IGHE"],
                 "Melanoma": ["MLANA", 'PRAME', 'SOX10', "S100A1"],
                 }

sc.pl.matrixplot(
    adata_mm,
    BCELL_MM_GENES,
    groupby="celltypes_sFig4_melanoma",
  # standard_scale="var",
    vmax=0.5,cmap="Reds",
    vmin=0,
    dendrogram=False,
    
    colorbar_title="Mean expression\n(log1p norm)",
 

) 


sc.pl.matrixplot(
    adata_mm,
    BCELL_MM_GENES,
    groupby="celltypes_sFig4_melanoma",
      standard_scale="var",
    vmax=1,cmap="Reds",
    vmin=0,
    dendrogram=False,
    
    colorbar_title="Mean expression\n(variance scaled)",
 

) 

# Fig 7

In [ ]:
sc.settings.figdir = "fig7"


In [ ]:
KEEP_5k_new = [F23_NAME,
 'Other immune cell',
 'F2: Universal',
 'VE/Pericyte',
 'F3: FRC-like',
 F1_NAME,
 'Schwann',
 'F4: Mixed',
 'F5: Schwann-like',
 'F4: TNN+COCH+', 'MigDC', 'Tcell', 'T']

# color_map_5k_new = {'F3: FRC-like': '#FF0000',
#  F23_NAME: '#0000FF',
#  'F5: Schwann-like': '#7850b2',
#  'VE/Pericyte': '#696969',
#  'F2: Universal': '#ADD8E6',
#  F1_NAME: '#FFFF00',
#                      'F1: Regenerative': '#FFFF00',
#  'Schwann': '#000000',
#  'F4: Mixed': '#008000',
#  'F4: TNN+COCH+': '#80ff80',
#  'Other': '#DCDCDC',
#  'Other immune cell': '#D2B48C',
#     "T": "#654321",
#                     "MigDC": "#00FF00"
                   
                   #}
mm_colors2= {
    'T': '#c3aa85',                
    'Mac': '#FFA500',              
    'cDC': '#3CB371',               
    'MigDC': '#8B4513',             
    #F1_NAME: '#FFFF00', 
}

# mm_colors2 =  {
#     "Bcell/plasma cell": "#FF69B4",
#     "MigDC": "#00FF00",
#     "Mast cell":  '#ffaf2b',
#       'Tcell': "#DEB887", #'#ffcdd7',
#     'T': '#DEB887',
#         'LE': '#800020',
#   'KC': '#DCDCDC',


# }
color_map_5k_new = color_map_5k | mm_colors2

sc.settings.set_figure_params(dpi=300, facecolor="white", frameon=False, figsize=(13,13))

CATEGORY="celltypes_Fig7"

adata_5k.obs["celltypes_Fig7"] = [x if x in KEEP_5k_new else "Other" for x in adata_5k.obs[CATEGORY]]

adata_5k.obs["celltypes_Fig7"] =  adata_5k.obs["celltypes_Fig7"].astype('category')
adata_5k.uns['celltypes_Fig7_colors'] = [color_map_5k_new[x] for x in adata_5k.obs["celltypes_Fig7"].cat.categories]
 

for SAMPLE_INDEX, sample in enumerate(["Lesional_atopicdermatitis3"]):
    if "Lesional" in sample:

        adata_eczema = adata_5k[adata_5k.obs["sample_id"] == "Lesional_atopicdermatitis3"]
        category_counts = adata_eczema.obs['celltypes_Fig7'].value_counts()

        categories_to_keep = category_counts[category_counts >= 10].index.tolist()
        adata_eczema = adata_eczema[adata_eczema.obs['celltypes_Fig7'].isin(categories_to_keep)].copy()


#         try:
#             adata_eczema.uns.pop('celltypes_Fig7_colors')
#         except:
#             1 

        sq.pl.spatial_scatter(
            adata_eczema,
            library_id="spatial",
            shape=None,
            color="celltypes_Fig7",
            title=sample,
            size=5,
            #save="_fig6umap_xenium_withbox.png"
        )

        ax = plt.gca()
        plt.show()

In [ ]:
custom_colors = {
    "Perivascular superficial": "#ffb6c1", 
    "Other": "#d3d3d3"  
}

adata_eczema.obs["Niche_name"] = adata_eczema.obs["Niche_name"].astype("category")

adata_eczema.uns["Niche_name_colors"] = [
    custom_colors.get(cat, "#d3d3d3")   
    for cat in adata_eczema.obs["Niche_name"].cat.categories
]

sq.pl.spatial_scatter(
    adata_eczema,
    library_id="spatial",
    shape=None,
    color="Niche_name",
    title=sample,
    size=5,
   # save="_FIG6_EICHR.png"
)

plt.show()

In [ ]:
adata_eczema_niche=adata_eczema[adata_eczema.obs["Niche_name"].str.startswith("Periv")]
adata_eczema_niche.obs.celltypes_Fig7.value_counts()

In [ ]:
# color_map_5k = {
#     'F3: FRC-like': '#FF0000',
#     F23_NAME: '#0000FF',
#     'F5: Schwann-like': "#7850b2",
#     'VE/Pericyte': '#696969',
#     'F2: Universal': '#ADD8E6',
#     F1_NAME: '#FFFF00',
#       'F1: Regenerative' : '#FFFF00',
# 'F1: Regenerative': '#FFFF00',
#     'Schwann': '#000000',
#     'F4: Mixed': '#008000',
#     'F4: TNN+COCH+': '#80ff80',
#     'Other': '#DCDCDC',
#     'Immune cell': "#D2B48C",
#     'LE': '#800020',
#     'KC': '#DCDCDC',
#         'Melanoma': '#2F2F2F',
#     'F6: Myofibroblast inflammatory': '#01ffff',  # cyan
#       'F6: Myofibroblast': '#55aaff',  # light blue
#     "Bcell/plasma cell": "#FF69B4",
#     "MigDC": "#00FF00",
#     "Mast cell":  '#ffaf2b',
#       'Tcell': "#DEB887", #'#ffcdd7',
#     'T': '#DEB887',

# }


# color_map_5k = {
#     'F3: FRC-like': "#FF1493",#  '#FF0000', 
#     F23_NAME: '#0000FF',
#     #'F2 Universal/F1': '#4d4d4d',

#     'F5: Schwann-like': "#7850b2", #'#800080',
#     'VE/Pericyte': "#FFC1C1", #"#FFC0CB",#'#696969', #'#A9A9A9',  "#FF0000", 
#     'F2: Universal': '#ADD8E6',
#     #'F2 Universal/F1': '#0000FF',
    
#     "F1: Superficial": '#FFFF00',    'F1: Regenerative': '#FFFF00',

#     'Schwann': '#000000',
    
#     # Updated F4 colors
#     'F4: DS_DPEP1+': '#008000',         # Dark green
#     #'F4: Mixed': '#',      # Mid green (standard green)
#     'F4: TNN+COCH+': '#80ff80',     # Very light green
    
#     'Other': '#DCDCDC', #'lightgrey'
    
#     'Immune cell': "#D2B48C"
# }
# adata_5k.uns['celltypes_Fig2_colors'] = [color_map_5k[x] for x in adata_5k.obs["celltypes_Fig2"].cat.categories]



In [ ]:
category_counts = adata_eczema_niche.obs['celltypes_Fig7'].value_counts()

categories_to_keep = category_counts[category_counts >= 20].index.tolist()


adata_eczema_niche = adata_eczema_niche[adata_eczema_niche.obs['celltypes_Fig7'].isin(categories_to_keep)].copy()
categories = adata_eczema_niche.obs['celltypes_Fig7'].cat.categories


default_color = '#808080'   
colors = [color_map_5k.get(cat, default_color) for cat in categories]

for cat, color in zip(categories, colors):
    print(f"Category: {cat}, Color: {color}")

adata_eczema_niche.uns['celltypes_Fig7_colors'] = colors


sq.pl.spatial_scatter(
    adata_eczema_niche,
    library_id="spatial",
    shape=None,
    color="celltypes_Fig7",
    title='' ,
    size=30,    edgecolor='black',       
    linewidth=0.1    ,
)
plt.show()

In [ ]:
proportions = adata_eczema_niche.obs['celltypes_Fig7'].value_counts(normalize=True)


fig, ax = plt.subplots(figsize=(6, 1.5))   

previous_proportion = 0
for proportion, label in zip(proportions, proportions.index):
    ax.barh([1], [proportion], left=previous_proportion, color=color_map_5k[label], label=label,
                        linewidth=0.5,   
           edgecolor='black', 
           )
    previous_proportion += proportion

ax.set_yticks([])   
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)


ax.set_xticks([0, 0.25, 0.5, 0.75, 1.0])
ax.set_xticklabels(['0', '0.25', '0.5', '0.75', '1'])
ax.set_xlabel('Proportion')


ax.grid(False)


plt.tight_layout()


plt.show()

In [ ]:
proportions_filtered = proportions[proportions.index != 'Nonspecific']

other_proportion = proportions_filtered[proportions_filtered < 0.01].sum()
proportions_filtered = proportions_filtered[proportions_filtered >= 0.01]

if other_proportion > 0:
    proportions_filtered['Other'] = other_proportion

proportions_filtered = proportions_filtered / proportions_filtered.sum()
proportions_filtered = proportions_filtered.sort_values(ascending=False)
fig, ax = plt.subplots(figsize=(1, 6)) 
previous_proportion = 0
for proportion, label in zip(proportions_filtered, proportions_filtered.index):
    color = color_map_5k.get(label, '#D3D3D3')   
    ax.bar([0.5], [proportion], bottom=previous_proportion, color=color, width=0.8, label=label)
    previous_proportion += proportion

handles, labels = ax.get_legend_handles_labels()
unique_labels = dict(zip(labels, handles))   
ax.legend(unique_labels.values(), unique_labels.keys(), loc='upper left', bbox_to_anchor=(1, 1),
          frameon=False)

ax.set_xticks([])
ax.set_yticks([0, 0.25, 0.5, 0.75, 1.0])
ax.set_yticklabels(['0', '0.25', '0.5', '0.75', '1'])
ax.set_ylabel('Proportion')
ax.set_xlabel('')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 1.5))  

previous_proportion = 0
for proportion, label in zip(proportions, proportions.index):
    ax.barh([1], [proportion], left=previous_proportion, color=color_map_5k[label], label=label,
                        linewidth=0.5,   
      edgecolor='black', )
    previous_proportion += proportion

ax.set_yticks([])  
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax.set_xticks([0, 0.25, 0.5, 0.75, 1.0])
ax.set_xticklabels(['0', '0.25', '0.5', '0.75', '1'])
ax.set_xlabel('Proportion')
ax.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
file_path = '/nfs/team298/ls34/fb_dict.pkl'
with open(file_path, 'rb') as file:
    reynolds_short_final = pickle.load(file)

print(f"Dictionary successfully LOADED from {file_path}")

reynolds_short_final = {key: [gene for gene in genes if gene in adata_eczema_niche.var_names] 
                           for key, genes in reynolds_short_final.items()}

adata_eczema_niche2 = adata_eczema_niche[adata_eczema_niche.obs["celltypes_Fig7"].str.startswith("F")]

adata_eczema2 = adata_eczema[adata_eczema.obs["celltypes_Fig7"].str.startswith("F")]
sc.pl.dotplot(
    adata_eczema2,
    reynolds_short_final,
    groupby="celltypes_Fig7",
    standard_scale="var",
    dendrogram=False,
    colorbar_title="Mean expression\n(variance scaled)",
    categories_order=  ['F1: Superficial', 'F2: Universal', 'F2/3: Stroma_PPARG+', 'F3: FRC-like', #'F4: DS_DPEP1+',
                        'F5: Schwann-like',]

) 
FIG6_GENES= {              "MigDC": ["CCR7", "LAMP3", "CD1B", "EBI3"],

    "T": ["CD3E", "GATA3", "CXCR4", 'CD96'],
                    "VE/Peri": ["CLDN5", "PECAM1", "SOX18", "RGS5"],

              }
adata_eczema2 = adata_eczema[~adata_eczema.obs["celltypes_Fig7"].str.startswith("F")]

sc.pl.dotplot(
    adata_eczema2,
    FIG6_GENES,
    groupby="celltypes_Fig7",
    standard_scale="var",
    dendrogram=False,
   # swap_axes=True,
    colorbar_title="Mean expression\n(variance scaled)"
) 

In [ ]:
adata_eczema2 = adata_eczema[adata_eczema.obs["celltypes_Fig7"].str.startswith("F")]



In [ ]:
file_path = '/nfs/team298/ls34/fb_dict.pkl'
with open(file_path, 'rb') as file:
    reynolds_short_final = pickle.load(file)

print(f"Dictionary successfully LOADED from {file_path}")

reynolds_short_final = {key: [gene for gene in genes if gene in adata_eczema_niche.var_names] 
                           for key, genes in reynolds_short_final.items()}


adata_eczema2 = adata_eczema[adata_eczema.obs["celltypes_Fig7"].str.startswith("F")]
sc.pl.dotplot(
    adata_eczema2,
    reynolds_short_final,
    groupby="celltypes_Fig7",
    standard_scale="var",
    dendrogram=False,
    colorbar_title="Mean expression\n(variance scaled)"
) 
adata_eczema2 = adata_eczema[~adata_eczema.obs["celltypes_Fig7"].str.startswith("F")]
sc.pl.dotplot(
    adata_eczema2,
    reynolds_short_final,
    groupby="celltypes_Fig7",
    standard_scale="var",
    dendrogram=False,
    #swap_axes=True,
    colorbar_title="Mean expression\n(variance scaled)"
) 

In [ ]:
# proportions_filtered = proportions[proportions.index != 'Nonspecific']
# other_proportion = proportions_filtered[proportions_filtered < 0.01].sum()
# proportions_filtered = proportions_filtered[proportions_filtered >= 0.01]
# if other_proportion > 0:
#     proportions_filtered['Other'] = other_proportion
# proportions_filtered = proportions_filtered / proportions_filtered.sum()
# proportions_filtered = proportions_filtered.sort_values(ascending=False)
# fig, ax = plt.subplots(figsize=(1, 6))  

# previous_proportion = 0
# for proportion, label in zip(proportions_filtered, proportions_filtered.index):
#     color = color_map_5k.get(label, '#D3D3D3') 
#     ax.bar([0.5], [proportion], bottom=previous_proportion, color=color, width=0.8, label=label)
#     previous_proportion += proportion

# handles, labels = ax.get_legend_handles_labels()
# unique_labels = dict(zip(labels, handles))  
# ax.legend(unique_labels.values(), unique_labels.keys(), loc='upper left', bbox_to_anchor=(1, 1),
#           frameon=False)

# ax.set_xticks([])
# ax.set_yticks([0, 0.25, 0.5, 0.75, 1.0])
# ax.set_yticklabels(['0', '0.25', '0.5', '0.75', '1'])
# ax.set_ylabel('Proportion')
# ax.set_xlabel('')

# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)
# ax.spines['left'].set_visible(False)
# ax.spines['bottom'].set_visible(False)

# ax.grid(False)
# plt.tight_layout()
# plt.show()

In [ ]:
CATEGORY="celltypes_Fig7"
adata_i=adata_eczema_niche
 



In [ ]:
FIG6_GENES= {       "F1": ["COL18A1", 'COL23A1', "APCDD1"],
             "F2+F2/3": ["CXCL12", "MFAP5", "DPP4"],
                     "F3": ["CCL19", "CH25H", "IL33", "TNFSF13B","IL15", "VCAM1"],
             
 
              }
    
adata_i2=adata_i[adata_i.obs[CATEGORY].str.startswith("F")]
sc.pl.dotplot(
    adata_i2,
    FIG6_GENES,
    groupby=CATEGORY,
    standard_scale="var",
    dendrogram=False,
)  
FIG6_GENES= {      
             "VE/Peri": ["CLDN5", "PECAM1", "SOX18", "RGS5"],
    "T": ["CD3E", "GATA3", "CXCR4", 'CD96'],
        "MigDC": ["CCR7", "LAMP3", "CD1B", "EBI3"],
       
              }
adata_i2=adata_i[~adata_i.obs[CATEGORY].str.startswith("F")]
sc.pl.dotplot(
    adata_i2,
    FIG6_GENES,
    groupby=CATEGORY,
    standard_scale="var",
    dendrogram=False,
)  

In [ ]:
FIG6_GENES= {       "F1": ["COL18A1", 'COL23A1', "APCDD1"],
             "F2+F2/3": ["CXCL12", "MFAP5", "DPP4"],
                     "F3": ["CCL19", "CH25H", "IL33", "TNFSF13B","IL15", "VCAM1"],
             
 
              }
    
adata_i2=adata_i[adata_i.obs[CATEGORY].str.startswith("F")]
sc.pl.matrixplot(
    adata_i2,
    FIG6_GENES,
    groupby=CATEGORY,
   standard_scale="var",
    dendrogram=False,
    cmap="Reds",# vmax=0.7
    colorbar_title="Mean expression\n(variance scaled)",
   # save="fig6_dotplot1.png"
    
)  


FIG6_GENES= {              "MigDC": ["CCR7", "LAMP3", "CD1B", "EBI3"],

    "T": ["CD3E", "GATA3", "CXCR4", 'CD96'],
                    "VE/Peri": ["CLDN5", "PECAM1", "SOX18", "RGS5"],

              }
adata_i2=adata_i[~adata_i.obs[CATEGORY].str.startswith("F")]
sc.pl.matrixplot(
    adata_i2,
    FIG6_GENES,
    groupby=CATEGORY,
    standard_scale="var",
    dendrogram=False,
        cmap="Reds",# vmax=0.7
        colorbar_title="Mean expression\n(variance scaled)",
        #save="fig6_dotplot2.png",

    categories_order = ["MigDC", "T", "VE/Pericyte", "Other"]


)  

In [ ]:
sc.pl.dotplot(
    adata_i,
    reynolds_short_final,
    groupby="celltypes_Fig7",
    standard_scale="var",
    dendrogram=False,
    colorbar_title="Mean expression\n(variance scaled)"
)  

In [ ]:
# adata_box=adata_5k[adata_5k.obs["Niche_name"]=="Perivascular superficial"]
# adata_box.obs["celltypes_Fig2"] = adata_box.obs["celltypes_Fig2"].astype(str)
# adata_box = adata_box[adata_box.obs['celltypes_Fig2'].map(adata_box.obs['celltypes_Fig2'].value_counts()) > 50]
# group_counts = adata_box.obs["celltypes_Fig2"].value_counts()
# custom_colors2 = {
#        F1_NAME: np.array([1.0, 1.0, 0.89803922, 1.0]),
#     'F2: Universal': np.array([0.81411765, 0.88392157, 0.94980392, 1.0]),
#     F23_NAME: np.array([0.47294118, 0.71163399, 0.85071895, 1.0]),
#     'F3: FRC-like': np.array([0.99692426, 0.89619377, 0.84890427, 1.0]),
#     'F4: DS_DPEP1+': np.array([0.82745098, 0.93254902, 0.80313725, 1.0]),
#     'F4: TNN+COCH+': np.array([0.50196078, 0.79058824, 0.50143791, 1.0]),
#     'F4: DP_HHIP+': np.array([0.15294118, 0.56183007, 0.28313725, 1.0]),
#     'F5: RAMP1+': np.array([0.6177624, 0.60213764, 0.78345252, 1.0]),
#     'F5: NGFR+': np.array([0.47320261, 0.43267974, 0.69934641, 1.0]),
#     'mSchwann': np.array([0.29411765, 0.0, 0.50980392, 1.0]),  # Dark purple
#     'nmSchwann': np.array([0.2, 0.0, 0.4, 1.0]),               # Darker purple
#     'Pericyte': np.array([0.54509804, 0.0, 0.0, 1.0]),         # Dark red
#     'F_Fascia': [0, 0, 0, 0.3]      ,                           # Semi-transparent black
#         'VE/Pericyte': '#696969',
#    # 'F2: Universal': '#ADD8E6',
#     #F1_NAME: '#FFFF00',
#      #'F1: Regenerative' : '#FFFF00',
#     'Schwann': '#000000',
#     #'F4: Mixed': '#008000',
#     #'F4: TNN+COCH+': '#80ff80',
#     'Other': '#DCDCDC',
#     'Immune cell': "#D2B48C"
# }

# colors = [custom_colors2.get(group, '#D3D3D3') for group in group_counts.index]
# plt.figure(figsize=(3, 3))   
# group_counts.plot(kind='bar', color=colors, width=0.8)  


# plt.ylabel("Cell count")           
# plt.xlabel("")                     
# plt.xticks(rotation=90, ha='right', fontsize=10)   
# plt.yticks(fontsize=10)             
# plt.grid(False)                     
# plt.title("")                    


# plt.gca().spines['top'].set_visible(False)
# plt.gca().spines['right'].set_visible(False)
# plt.tight_layout()
# plt.show()

